In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Set the global policy to mixed precision with float16
mixed_precision.set_global_policy('mixed_float16')

# List all physical devices available
devices = tf.config.list_physical_devices()
print("\nDevices: ", devices)

# List all available GPU devices
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # If GPUs are available, set memory growth to True for each GPU
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
# Optimización para el data pipeline
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 16
IMG_SIZE = 512

In [ ]:
# Directorio de imágenes
base_dir = 'Images'

train_datagen = ImageDataGenerator(
    rescale=1.0/255, 
    validation_split=0.2, 
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

In [ ]:
def make_classifier_model():
    # Initialize a sequential model
    model = tf.keras.Sequential()
    
    # Add first convolutional layer with 64 filters, 3x3 kernel, same padding, and input shape
    model.add(layers.Conv2D(64, (3, 3), padding='same', input_shape=(512, 512, 3)))
    # Add batch normalization
    model.add(layers.BatchNormalization())
    # Add LeakyReLU activation function with alpha 0.1
    model.add(layers.LeakyReLU(alpha=0.1))
    # Add max pooling layer with 2x2 pool size
    model.add(layers.MaxPooling2D((2, 2)))
    # Add dropout layer with 30% dropout rate
    model.add(layers.Dropout(0.3))
    
    # Add second convolutional layer with 128 filters, 3x3 kernel, and same padding
    model.add(layers.Conv2D(128, (3, 3), padding='same'))
    # Add batch normalization
    model.add(layers.BatchNormalization())
    # Add LeakyReLU activation function with alpha 0.1
    model.add(layers.LeakyReLU(alpha=0.1))
    # Add max pooling layer with 2x2 pool size
    model.add(layers.MaxPooling2D((2, 2)))
    # Add dropout layer with 30% dropout rate
    model.add(layers.Dropout(0.3))
    
    # Add third convolutional layer with 256 filters, 3x3 kernel, and same padding
    model.add(layers.Conv2D(256, (3, 3), padding='same'))
    # Add batch normalization
    model.add(layers.BatchNormalization())
    # Add LeakyReLU activation function with alpha 0.1
    model.add(layers.LeakyReLU(alpha=0.1))
    # Add max pooling layer with 2x2 pool size
    model.add(layers.MaxPooling2D((2, 2)))
    # Add dropout layer with 30% dropout rate
    model.add(layers.Dropout(0.3))
    
    # Add fourth convolutional layer with 512 filters, 3x3 kernel, and same padding
    model.add(layers.Conv2D(512, (3, 3), padding='same'))
    # Add batch normalization
    model.add(layers.BatchNormalization())
    # Add LeakyReLU activation function with alpha 0.1
    model.add(layers.LeakyReLU(alpha=0.1))
    # Add max pooling layer with 2x2 pool size
    model.add(layers.MaxPooling2D((2, 2)))
    # Add dropout layer with 30% dropout rate
    model.add(layers.Dropout(0.3))
    
    # Flatten the output from the convolutional layers
    model.add(layers.Flatten())
    # Add a dense (fully connected) layer with 256 units
    model.add(layers.Dense(256))
    # Add batch normalization
    model.add(layers.BatchNormalization())
    # Add LeakyReLU activation function with alpha 0.1
    model.add(layers.LeakyReLU(alpha=0.1))
    # Add dropout layer with 50% dropout rate
    model.add(layers.Dropout(0.5))
    # Add output dense layer with 1 unit and sigmoid activation function
    model.add(layers.Dense(1, activation='sigmoid', dtype='float32'))
    
    # Return the constructed model
    return model

In [ ]:
model = make_classifier_model()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
if not os.path.exists('models'):
    os.makedirs('models')

In [ ]:
# Define a list of callbacks to be used during training
callbacks = [
    # Reduce learning rate when a metric has stopped improving
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',  # Metric to be monitored
        factor=0.2,          # Factor by which the learning rate will be reduced
        patience=2           # Number of epochs with no improvement after which learning rate will be reduced
    ),
    # Save the model after every epoch
    tf.keras.callbacks.ModelCheckpoint(
        'models/checkpoint2.keras',  # Path to save the model file
        save_best_only=True          # Only save the model if the monitored metric improves
    )
]

In [ ]:
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
if not os.path.exists('saved_models'):
    os.makedirs('saved_models')

model.save('saved_models/glasses_detector')

if 'history' in locals() or 'history' in globals():
    history_dict = history.history
    with open('saved_models/training_history.json', 'w') as f:
        json.dump(history_dict, f)